In [1]:

import pandas as pd
import pandasql as ps
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.diagnostic import linear_rainbow, het_breuschpagan
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.stattools import jarque_bera

from sklearn.preprocessing import LabelEncoder




In [10]:
dfAll = pd.read_csv("../references/dfAll.csv")

In [11]:
# Taking only Single Family Residences between 50000 and 1000000
q2 = ("""
SELECT * 
FROM dfAll
WHERE PropertyType = 11 and SalePrice > 50000 and SalePrice < 1000000
""")
dfSingleFamily =ps.sqldf(q2)

In [22]:
dfSingleFamily.columns

Index(['MajorMinor', 'SalePrice', 'PropertyType', 'NbrLivingUnits', 'Address',
       'BuildingNumber', 'StreetName', 'StreetType', 'ZipCode', 'Stories',
       'BldgGrade', 'BldgGradeVar', 'SqFt1stFloor', 'SqFtHalfFloor',
       'SqFt2ndFloor', 'SqFtUpperFloor', 'SqFtUnfinFull', 'SqFtTotLiving',
       'SqFtTotBasement', 'SqFtFinBasement', 'FinBasementGrade',
       'SqFtGarageBasement', 'SqFtGarageAttached', 'DaylightBasement',
       'SqFtOpenPorch', 'SqFtEnclosedPorch', 'SqFtDeck', 'HeatSystem',
       'HeatSource', 'ViewUtilization', 'Bedrooms', 'BathHalfCount',
       'Bath3qtrCount', 'BathFullCount', 'FpSingleStory', 'FpMultiStory',
       'FpFreestanding', 'YrBuilt', 'YrRenovated', 'Condition', 'WfntLocation',
       'SqFtLot', 'TrafficNoise', 'AirportNoise', 'PowerLines',
       'OtherNuisances'],
      dtype='object')

In [66]:
ols(formula="SalePrice ~ SqFtTotLiving", data=dfSingleFamily).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.160
Model:                            OLS   Adj. R-squared:                  0.160
Method:                 Least Squares   F-statistic:                     2996.
Date:                Wed, 04 Mar 2020   Prob (F-statistic):               0.00
Time:                        12:27:16   Log-Likelihood:            -2.1196e+05
No. Observations:               15704   AIC:                         4.239e+05
Df Residuals:                   15702   BIC:                         4.239e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept      3.898e+05   4027.820     96.766      0.000    3.82e+05    3.98e+05
SqFtTotLiving   110.5109      2.019     54.736      0.000     106.553     114.468
==============================================================================
Omnibus:                     1059.199   Durbin-Watson:                   2.005
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              487.046
Skew:                           0.237   Prob(JB):                    1.73e-106
Kurtosis:                       2.279   Cond. No.                     5.72e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.72e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [60]:
q3 = ("""
SELECT SalePrice, BldgGrade,
CASE
    WHEN SqFtGarageAttached > 0 THEN 1
    ELSE 0
END AS garageattached
FROM dfSingleFamily
""")
dfGarage =ps.sqldf(q3)

In [62]:
ols(formula="BldgGrade ~  garageattached", data=dfGarage).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              BldgGrade   R-squared:                       0.100
Model:                            OLS   Adj. R-squared:                  0.100
Method:                 Least Squares   F-statistic:                     1747.
Date:                Wed, 04 Mar 2020   Prob (F-statistic):               0.00
Time:                        12:14:19   Log-Likelihood:                -19872.
No. Observations:               15704   AIC:                         3.975e+04
Df Residuals:                   15702   BIC:                         3.976e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          7.0649      0.010    713.665      0.000       7.045       7.084
garageattached     0.5727      0.014     41.795      0.000       0.546       0.600
==============================================================================
Omnibus:                      293.191   Durbin-Watson:                   2.003
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              382.333
Skew:                           0.249   Prob(JB):                     9.49e-84
Kurtosis:                       3.580   Cond. No.                         2.67
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""